In [2]:
from guess_number import guess_number
from numpy import random, mean

def fix_len_num(num: int, length: int) -> str:
    num = str(num)
    return '0'*(length-len(num)) + num 

def average_int_result(func, repeat=10000, print_chart=False):
    """Decorator for 

    Args:
        func (_type_): _description_
        repeat (int, optional): _description_. Defaults to 10000.
        print_chart (bool, optional): _description_. Defaults to False.
    """
    def decorator(func):
        def dec_func(*args, **kwargs):
            results = [func(*args, **kwargs) for i in range(repeat)]
            avrg_res = round(mean(results))
            
            if print_chart:
                # Printing a simple chart in terminal
                min_res = min(results) # lower bound
                max_res = max(results) # upper bound
                results = {i: results.count(i)
                            for i in range(min_res, max_res+1)}
                for result in results.items():
                    max_val = max(results.values())
                    print(fix_len_num(result[0], 2), '-',
                            fix_len_num(result[1], 6), 
                            ('=' if result[0] == avrg_res else '-') 
                            * round(result[1]/max_val*30)) 
            return avrg_res
        return dec_func
    return decorator(func)

guess_number = average_int_result(guess_number, print_chart=True)
for hi in [10**i for i in range(2,6)]:
    print(f"\n1 <= number <= {hi}")
    print(f"Average number of tries is {guess_number(1, hi, bnr_srch=True)}")


1 <= number <= 100
01 - 000093 -
02 - 000206 --
03 - 000395 ---
04 - 000852 -------
05 - 001626 -------------
06 - 003108 =========================
07 - 003720 ------------------------------
Average number of tries is 6

1 <= number <= 1000
01 - 000007 
02 - 000030 
03 - 000030 
04 - 000079 
05 - 000144 -
06 - 000325 --
07 - 000632 ----
08 - 001290 --------
09 - 002527 ===============
10 - 004936 ------------------------------
Average number of tries is 9

1 <= number <= 10000
01 - 000001 
02 - 000002 
03 - 000003 
04 - 000009 
05 - 000018 
06 - 000030 
07 - 000066 
08 - 000123 -
09 - 000270 --
10 - 000555 ----
11 - 000973 -------
12 - 002081 ===============
13 - 004055 ------------------------------
14 - 001814 -------------
Average number of tries is 12

1 <= number <= 100000
03 - 000001 
04 - 000001 
05 - 000003 
06 - 000004 
07 - 000004 
08 - 000012 
09 - 000027 
10 - 000052 
11 - 000105 -
12 - 000212 --
13 - 000402 ---
14 - 000769 -------
15 - 001590 --------------
16 - 003360 ==